In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
sp500_df = pd.DataFrame()
sp500_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_df = sp500_wiki[0]
sp500_list = sp500_df['Symbol'].tolist()
sp500_list.sort()
print(sp500_list)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DL

In [19]:
# Set timeframe to pull data (365 * years)
lookback = 365 * 1 
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=lookback)
print(end_date)
print(start_date)

2020-09-15 11:54:18.022159
2019-09-16 11:54:18.022159


In [20]:
# Pull daily close information for S&P500
sp500_full=yf.download(sp500_list,start_date,end_date)['Adj Close']
sp500_full.head(-1)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
                     A        AAL         AAP        AAPL       ABBV  \
Date                                                                   
2019-09-16   76.466560  27.578360  154.972519   54.434902  65.843224   
2019-09-17   75.723198  28.442356  154.972519   54.632942  66.794762   
2019-09-18   76.882843  28.402632  152.708511   55.145359  66.486908   
2019-09-19   77.378410  28.104702  151.784698   54.697304  66.860069   
2019-09-20   77.279297  27.796841  154.655518   53.897728  67.531746   
...                ...        ...         ...         ...        ...   
2020-09-09   97.930000  13.050000  154.350006  117.320000  91.629997   
2020-09-10   98.570000  13.010000  153.559998  113.489998  89.650002   
2020-09-11   98.599998  12.950000  154.589996  112.000000  89.699997   
2

In [21]:
import numpy as np
import os as os
import matplotlib.pyplot as plt
from pathlib import Path
#from backtesting import Backtest,Strategy
from datetime import datetime, timedelta

In [22]:
# Function for generic return (i.e. momentum)
def generic_return(start,end,data):
    # update formatting for the dates
    start_date = start.strftime("%Y-%m-%d")
    end_date = end.strftime("%Y-%m-%d")
    
    data = data[start:end]
    
    #calculate pct change during the period
    overall_returns = data.pct_change()
    
    #determine cumulative returns ending at the end of the period minus the last day
    cumulative_returns = (1 + overall_returns).cumprod() - 1
    
    #final total return for the period
    total_return = cumulative_returns.iloc[[-1]]
    
    #take the output and convert to a vertical list of tickers and their cumulative return with date
    return_list = total_return.stack()
    
    #reset the index 
    df = return_list.to_frame().reset_index()
    
    #set columns of 'symbol' and 'return'
    df.rename(columns ={'level_1':'symbol',0:"return"}, inplace = True)
    
    #drop the duplicate 'date' column now that index = 'date'
    df.drop(columns=['Date'], inplace=True)
    
    #set the column of 'symbol' to the index
    df.set_index(df['symbol'], inplace = True)
    
    #drop the extra symbol column
    df.drop(columns=['symbol'], inplace=True)
    
    #break into deciles and add the decile column
    df['decile_rank']=pd.qcut(df['return'],10,labels=False)
    
    #get top decile
    top_decile = df.loc[df['decile_rank'] == 9]
    
    return top_decile

In [23]:
sp500_full.index = sp500_full.index.tz_localize(None)

In [24]:
# Create the generic return portfolio

portfolio = generic_return(start_date,end_date,sp500_full)
portfolio

,return,decile_rank
symbol,,
AAPL,1.156208,9
ABMD,0.481474,9
ADBE,0.745794,9
ADSK,0.556062,9
ALB,0.480141,9
ALGN,0.905911,9
AMD,1.548887,9
AMZN,0.729769,9
ANSS,0.524592,9


In [25]:
# Take the tickers from the output from Generic momentum and the date range selected earlier to create the fip score over
# that range
def fip(start,end,top_decile,data):
    
    #reset the index on the Top Decile to get the tickers
    top_decile = top_decile.reset_index()
    ticker_top=list(top_decile.iloc[:,0])
    
    #filter the dataset upon the date range provided
    start_date = start.strftime("%Y-%m-%d")
    end_date = end.strftime("%Y-%m-%d")
    data = data[start:end]
    
    # filter the dataset upon the tickers provided
    # data = data[ticker_top]
    data = data[[s for s in top_decile.symbol if s in data.columns]]   
    
    # pass the ticker_top list and the date range to select the portion of data to run the Fip_Score
    
    # Calculating the pct_change for the fip calculation
    data = data.pct_change().dropna()
    
    #initialize an empty library to hold the calculated Fip Score
    fip_scores = []
    
    # Set up a for loop to set variables for the number of positive / negative / zero and number of data. 
    # Determine the cumulativ returns and set to a variable. 
    for column in data.columns:
        num_positive = len(data.loc[data[column] > 0])
        num_negative = len(data.loc[data[column] < 0])
        num_zero = len(data.loc[data[column] == 0])
        num_days = len(data)
        cum_returns = data[column].sum() 
        
        # Perform the Fip Score calculation
    
        fip_score = np.sign(cum_returns) * (num_negative / num_days - num_positive/num_days)
        
        # As the last step in the loop, append the fip scores to get the whole list
        fip_scores.append(fip_score)
    
    # create a pd.Series with the appended fip scores, set all to the variable FIP
    fip = pd.Series(fip_scores,index = data.columns)
    
    # set the name of the series to Fip
    fip.name = "fip"
    
    #create a top_decile variable to an empty data frame with the index set to symbol
    top_decile = top_decile.set_index('symbol')
    
    
    # This one is tough, I'll leave this in place as this is a more complicated approach than we've taken previously
    
    return top_decile.merge(fip,left_index = True, right_index = True).sort_values(['return','fip'],ascending=False)

In [28]:
# Call the Fip function, pass in paramaters & output is top decile returns of momentum with a score of the FIP score. 

fip_portfolio = fip(start_date,end_date,portfolio,sp500_full)
fip_portfolio.sort_values(['fip','return'],ascending=True)

,return,decile_rank,fip
symbol,,,
CDNS,0.563771,9,-0.256
LOW,0.529531,9,-0.240
BIO,0.530864,9,-0.240
CHTR,0.472539,9,-0.224
DHR,0.473269,9,-0.224
ALB,0.480141,9,-0.208
SNPS,0.543745,9,-0.208
ADSK,0.556062,9,-0.208
AMZN,0.729769,9,-0.208
